In [15]:
import os
from collections.abc import MutableMapping
from pathlib import Path

import matplotlib.cm as cm
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import yaml

from java_migration.analysis.utils import (
    MavenStatus,
    flatten,
    get_experiment_data,
    get_maven_logs,
    visualize_agent_trace,
)
from java_migration.eval.maven_build_verifier import MavenBuildVerifier
from java_migration.eval.smol_log_parser import parse_log
from java_migration.eval.utils import recover_safe_repo_name
from java_migration.utils import REPO_ROOT

In [16]:
experiment_path = Path(
    # "/Users/mayvic/Documents/git/java-migration-paper/data/experiments/2025-03-09/22-19-56-hopeful-wescoff"
    # "/Users/mayvic/Documents/git/java-migration-paper/data/experiments/2025-03-11/13-55-09-condescending-galileo"
    # "/Users/mayvic/Documents/git/java-migration-paper/data/experiments/2025-03-11/14-14-38-awesome-pasteur"
    # "/Users/mayvic/Documents/git/java-migration-paper/data/experiments/2025-03-11/15-14-25-loving-newton"
    # "/Users/mayvic/Documents/git/java-migration-paper/data/experiments/2025-03-11/16-15-09-tender-varahamihira"
    "/Users/mayvic/Documents/git/java-migration-paper/data/experiments/2025-03-11/17-59-17-recursing-leavitt"
)
exp_data = get_experiment_data(experiment_path)

In [17]:
def get_experiment_features(exp_data):
    exp_feats = []
    for repo_name, repo_data in exp_data.items():
        cur_feats = {"repo_name": repo_name}
        if "build_result" in repo_data and repo_data["build_result"] is not None:
            cur_feats.update(flatten(repo_data["build_result"]))
        exp_feats.append(cur_feats)
        if "agent_log" in repo_data:
            cur_feats["run_duration"] = np.sum([y.meta.duration for y in repo_data["agent_log"].steps])
            cur_feats["input_tokens"] = np.sum([y.meta.input_tokens for y in repo_data["agent_log"].steps])
            cur_feats["output_tokens"] = np.sum([y.meta.input_tokens for y in repo_data["agent_log"].steps])
            cur_feats["num_steps"] = len(repo_data["agent_log"].steps)
            if len(repo_data["agent_log"].steps) > 1:
                if repo_data["agent_log"].steps[-2].raw_text:
                    cur_feats["vertex_fail"] = (
                        "ServiceUnavailableError" in repo_data["agent_log"].steps[-2].raw_text
                        or "RateLimitError" in repo_data["agent_log"].steps[-2].raw_text
                    )
                else:
                    cur_feats["vertex_fail"] = False
            else:
                cur_feats["vertex_fail"] = False
        if "repo_feats" in repo_data:
            cur_feats.update(repo_data["repo_feats"])

    return exp_feats


exp_features = get_experiment_features(exp_data)
df = pd.DataFrame(exp_features)
df

,repo_name,build_result_build_success,build_result_test_results_errors,build_result_test_results_failures,build_result_test_results_skipped,build_result_test_results_tests_run,build_result_test_success,error,run_success,run_duration,...,output_tokens,num_steps,vertex_fail,build_tool,number_of_external_dependencies,number_of_java_files,number_of_lines_of_code,number_of_modules,number_of_unit_tests,build_result_test_results
0,DeemOpen/zkui,True,0.0,0.0,0.0,0.0,True,None,True,31.04,...,134507,7,False,maven,28,20,2222,1,0,NaN
1,EalenXie/spring-microservice-ddd,True,0.0,0.0,0.0,2.0,True,None,True,18.66,...,147887,8,False,maven,14,34,2053,1,1,NaN
2,ata4/disunity,True,0.0,0.0,0.0,328.0,True,None,True,121.73,...,14046048,38,False,maven,12,95,7250,3,6,NaN
3,EnterpriseQualityCoding/FizzBuzzEnterpriseEdition,False,NaN,NaN,NaN,NaN,None,None,True,114.62,...,4294836,28,False,maven,12,87,2489,1,1,NaN
4,alibaba/QLExpress,True,0.0,0.0,4.0,386.0,True,None,True,167.17,...,29299289,33,False,maven,10,142,10938,1,193,NaN
5,SPuerBRead/Bridge,False,NaN,NaN,NaN,NaN,None,None,True,48.29,...,453650,13,False,maven,16,32,2408,1,0,NaN
6,Ouyangan/hunt-admin,True,NaN,NaN,NaN,NaN,True,None,True,48.40,...,4063859,20,False,maven,95,77,10788,6,59,NaN
7,anjia0532/unidbg-boot-server,True,NaN,NaN,NaN,NaN,True,None,True,24.37,...,244827,9,False,maven,30,9,619,1,8,NaN
8,nydiarra/springboot-jwt,True,0.0,0.0,0.0,0.0,True,None,True,63.33,...,1484942,12,False,maven,22,16,557,1,1,NaN


In [22]:
# exp_data["EalenXie/spring-microservice-ddd"]["agent_log"].steps
print(len(df))
print(df.build_result_build_success.sum())
print(df.build_result_test_success.sum())

9
7
7


In [25]:
# sns.violinplot(x="build_result_test_success", y="number_of_lines_of_code", data=df)